In [6]:
# ✅ Step 1: Import Libraries
# https://www.kaggle.com/datasets/msambare/fer2013/data
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# ✅ Step 2: Set Data Paths
data_dir = "D:/pcdownload/facial/train"
validation_dir = "D:/pcdownload/facial/test"

In [8]:
# ✅ Step 3: 预处理数据
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# 加载训练数据 (使用 train/)
train_generator = datagen.flow_from_directory(
    data_dir,  # 训练数据路径
    target_size=(48, 48),  # 统一大小
    color_mode='grayscale',  # 处理灰度图像
    batch_size=32,  # 每次训练的批量
    class_mode='categorical',  # 多类别分类
    subset='training'  # 80% 作为训练集
)

# 加载验证数据 (同样来自 train/，但属于 20% 验证集)
validation_generator = datagen.flow_from_directory(
    data_dir,  # 验证数据路径
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # 20% 作为验证集
)


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [9]:
# ✅ Step 4: Build CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion categories
])

In [10]:
# ✅ Step 5: Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Step 6: Train the Model
epochs = 20
history = model.fit(train_generator, validation_data=validation_generator, epochs=epochs)

# ✅ Step 7: Save Model
model.save("facial_expression_model.h5")

Epoch 1/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.2511 - loss: 1.8136 - val_accuracy: 0.3891 - val_loss: 1.5569
Epoch 2/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.3958 - loss: 1.5520 - val_accuracy: 0.4456 - val_loss: 1.4280
Epoch 3/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.4639 - loss: 1.3979 - val_accuracy: 0.5041 - val_loss: 1.3109
Epoch 4/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.4984 - loss: 1.3192 - val_accuracy: 0.5098 - val_loss: 1.2751
Epoch 5/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.5214 - loss: 1.2632 - val_accuracy: 0.5314 - val_loss: 1.2382
Epoch 6/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.5458 - loss: 1.1948 - val_accuracy: 0.5381 - val_loss: 1.2249
Epoch 7/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5636 - loss: 1.1550 - val_accuracy: 0.5410 - val_loss: 1.2159
Epoch 8/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5751 - loss: 1.1126 - val_acc

In [11]:
# ✅ Step 8: Test Model with All Images

test_folder = "D:/pcdownload/facial/test"
output_dir = "D:/pcdownload/facial/test_results"
os.makedirs(output_dir, exist_ok=True)  # 确保输出文件夹存在

# 获取类别标签
class_labels = list(train_generator.class_indices.keys())

# 为每种表情创建单独的 TXT 文件
output_files = {label: open(os.path.join(output_dir, f"{label}.txt"), "w", encoding="utf-8") for label in class_labels}

# 遍历测试集并预测
for category in os.listdir(test_folder):
    category_path = os.path.join(test_folder, category)
    if os.path.isdir(category_path):
        for img_name in os.listdir(category_path):
            image_path = os.path.join(category_path, img_name)
            if image_path.endswith((".jpg", ".png")):
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (48, 48))
                image = image / 255.0
                image = np.expand_dims(image, axis=0)
                image = np.expand_dims(image, axis=-1)
                prediction = model.predict(image)
                predicted_class = class_labels[np.argmax(prediction)]
                output_files[predicted_class].write(f"{image_path} - Predicted Expression: {predicted_class}\n")

# 关闭所有文件
for f in output_files.values():
    f.close()

print(f"✅ Test results saved in separate TXT files under {output_dir}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━